In [1]:
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"] # 设置字体
plt.rcParams["axes.unicode_minus"]=False # 正常显示负号
from database import * 

from pyecharts.charts import Bar, Line, Grid

import pyecharts.options as opts

In [2]:
sql = 'select score, count(*) count, avg(comment) avg_comment from movie group by score'
score, count, avg = execute_sql(sql, 3)

# plt.subplot(1, 2, 1)
# plt.bar(score, count, width=0.03)
# plt.title('评分-数量')

# plt.subplot(1, 2, 2)
# plt.plot(score, avg)
# plt.title('评分-平均评论数')

bar = (
        Bar()
        .add_xaxis(score)
        .add_yaxis('数量', count)
        .set_global_opts(
                title_opts=opts.TitleOpts(title='各评分的数量')
         )
)
line = (
        Line()
        .add_xaxis([str(i) for i in score])
        .add_yaxis('平均评论数', [round(i) for i in avg])
        .set_global_opts(
                title_opts=opts.TitleOpts(title='各评分对应的平均评论数', pos_top='52%'),
                legend_opts=opts.LegendOpts(pos_top='52%')
        )
)
grid = (
        Grid()
        .add(bar, opts.GridOpts(pos_bottom='55%'))
        .add(line, opts.GridOpts(pos_top='60%'))
)
grid.render_notebook()




In [3]:
sql =  'select name, count(*) count\
        from place p left outer join movie_place mp on p.id = mp.place_id\
        group by name\
        order by count desc'
name, count = execute_sql(sql, 2)

# plt.bar(name[:10], count[:10], width=0.5)
# plt.title('地区-数量')

bar = (
        Bar()
        .add_xaxis(name[:10])
        .add_yaxis('数量', count[:10])
        .set_global_opts(title_opts=opts.TitleOpts(title='各国家的入选数量', subtitle='前十名'))
)
bar.render_notebook()

In [4]:
sql = 'select p.name name, avg(score) avg_score\
        from place p left outer join movie_place mp on p.id = mp.place_id \
        left join movie m on m.id = mp.movie_id\
        group by p.name\
        having count(*) > 5\
        order by avg_score desc'
        
name, avg_score = execute_sql(sql, 2)
avg_score = [round(x, 2) for x in avg_score]
bar = (
        Bar()
        .add_xaxis(name)
        .add_yaxis('平均评分', avg_score)
        .set_global_opts(
                        yaxis_opts=opts.AxisOpts(min_=min(avg_score) - 0.01),
                        title_opts=opts.TitleOpts(title='各国家的平均得分', subtitle='大于五部电影计入统计')
        )
)
bar.render_notebook()

In [31]:
sql =  'select t.name name, count(*) count\
        from type t left outer join movie_type mt on t.id = mt.type_id \
        group by t.name \
        order by count desc'
        
name, count = execute_sql(sql, 2)
bar = (
        Bar()
        .add_xaxis(name)
        .add_yaxis('数量', count, category_gap="15%")
        .reversal_axis()
        .set_global_opts(
                        yaxis_opts=opts.AxisOpts(max_= 'dataMax', axislabel_opts=opts.LabelOpts(font_size=9,rotate=15)),
                        title_opts=opts.TitleOpts(title='各类型的电影数量', subtitle='')
        )
)
bar.render_notebook()